Irsyad Adam\
04/14/2022

In [5]:
#pip install neo4j
from neo4j import GraphDatabase
import pandas as pd

from tqdm import tqdm

In [4]:
nodes = {
        "name": ["Irsyad", "Adam", "Irsyad"], 
        "relationship": ["drinks", "likes", "eats"], 
        "object": ["water", "water", "pizza"]
        }

df = pd.DataFrame(nodes)
df[0:3]

,name,relationship,object
0,Irsyad,drinks,water
1,Adam,likes,water
2,Irsyad,eats,pizza


Link to the Neo4j Server

In [ ]:
driver = GraphDatabase.driver(uri = "bolt://localhost:7687",\
                              auth = ("neo4j","heart"))

Create Constraints for Uniqueness

In [ ]:
name_constraint = "CREATE CONSTRAINT ON (p:Person) ASSERT m.name IS UNIQUE"
object_constraint = "CREATE CONSTRAINT ON (o:object) ASSERT o.name IS UNIQUE"

with driver.session() as session:
    info = session.run(name_constraint)
    info = session.run(object_constraint)

Import Nodes

In [ ]:
def create_nodes(tx, name):
    query = "MERGE (p:Person{name: $name})"
    tx.run(query, name = name)

with driver.session() as session:
    for uid, name in tqdm(df["name"], desc="Importing Nodes"):
        session.write_transaction(create_nodes, name)

print("Success")

In [ ]:
def create_o_nodes(tx, name):
    query = "MERGE (p:Person{name: $name})"
    tx.run(query, name = name)

with driver.session() as session:
    for uid, name in tqdm(df["object"], desc="Importing object"):
        session.write_transaction(create_o_nodes, name)

print("Success")

In [ ]:
def create_relationship(tx, name, object, relationship):
    """
    :param name:
    :param object:
    :param relationship:
    """
    query = """
            MATCH (p:Person{name: $name})
            MATCH (o:object{name: $o_name})
            MERGE (p)-[{r: $relationship}]-(o)
            """
    tx.run(query, name = name, o_name = object, relationship = relationship)

with driver.session() as session:
    for name, o_name, rel in tqdm(zip(df["name"], df["object"], df["relationship"]), desc="Creating Relationships"):
        session.write_transaction(create_relationship, name, o_name, rel)
    
print("success")